# Permutation Test with nested family

For this example problem, we will just use a set of thickness ROI's, just family id as the block structure, and a single confounding variable of sex, and let's say neurocog1 as our target variable. Our dataset we are using is from the ABCD study.

### Load the data

In [31]:
import pandas as pd
import numpy as np

from neurotools.stats.permutations import permuted_v
from neurotools.loading.abcd import load_from_csv, load_family_block_structure

In [32]:
# Get possible columns
csv_loc = '/home/sage/benchmark_methods/data/nda3.0.csv'
all_cols = list(pd.read_csv(csv_loc, nrows=0))

# Load thickness ROI data - ignore hemi and full mean for plotting
thick = [col for col in all_cols if 'smri_thick_cort.destrieux' in col and '_mean' not in col]

# Rest of needed columns
rest = ['C(sex_at_birth)', 'neurocog_pc1.bl', 'C(rel_family_id)']

# Load in all together
data = load_from_csv(cols=thick+rest, csv_loc=csv_loc, drop_nan=True)

### Run the permutation test

So how should be use the rel_family_id variable with information on the unique family of each subject? One option would be to use the values directly to define groups and the families themselves, where if we set parameter within_grp to True, this would mean that swaps will only be performed within group, so in terms of family, that means the only swaps that would take place are within families, swapping siblings data with each other. What that also means is that we would not be allowing any swaps to take place between say two random subjects with no other family members (this likely is not what we want from a null model).

On the other hand, we could use the family information to define block-wise swap. That is to say, so every singleton family is freely swapped with other singleton families, but any family of size 2, can only be swapped with another family of size 2. In this case though, what happens internally further is that the data is split into different variance groups. The set of all single-tons for example all fall under the same unique variance group, then for the set of families of size two, we compute two new variance groups, reflecting the subjects data which are allowed to swap. So for example:

- sub1 - family 1
- sub2 - family 2 
- sub3 - family 2 
- sub4 - family 3 
- sub5 - family 3 

The only possible swap here is subj2 and subj3's values with subj4 and subj5 as a block-level swap. So the different automatically computed variance groups would be:

- sub1 - family 1 - vg = 1
- sub2 - family 2 - vg = 2
- sub3 - family 2 - vg = 3
- sub4 - family 3 - vg = 2
- sub5 - family 3 - vg = 3

Since sub2 and sub4 can swap, and sub3 and sub5 can swap, so they have the same variance group. This is imporant because in calculating the v-score, stats are computed for each variance group seperately, then summed. But are these the swaps and variance groups we want? It seems more desirable actually that we would allow another level of swaps to occur, that is to say, members of the same family should be allowed to swap too, right? Likewise, there is no real reason the first member of family 2 should be in a seperate variance group from the second member of family 2, instead we likely want them all to be in the same variance group. So something like this:

- sub1 - family 1 - vg = 1

- sub2 - family 2 - vg = 2
- sub3 - family 2 - vg = 2

- sub4 - family 3 - vg = 2
- sub5 - family 3 - vg = 2

Where now what we are explicitly concerned with is not the unique family, but the size of the family.



In [33]:
unique, cnts = np.unique(data['rel_family_id'], return_counts=True)
data['family_size'] = data['rel_family_id'].replace(dict(zip(unique, cnts)))
np.unique(data['family_size'])

array([1., 2., 3., 5.])

In [34]:
data['neg_rel_family_id'] = -data['rel_family_id']
data['ones'] = 1

permutation_structure = data[['family_size', 'neg_rel_family_id', 'ones']]
permutation_structure

,family_size,neg_rel_family_id,ones
src_subject_id,,,
NDAR_INV003RTV85,1.0,-6763.0,1
NDAR_INV005V6D2C,1.0,-7934.0,1
NDAR_INV007W6H7B,1.0,-3691.0,1
NDAR_INV00BD7VDC,1.0,-2923.0,1
NDAR_INV00CY2MDM,2.0,-4180.0,1
...,...,...,...
NDAR_INVZZL0VA2F,2.0,-7707.0,1
NDAR_INVZZLZCKAY,1.0,-7246.0,1
NDAR_INVZZPKBDAC,1.0,-1960.0,1


So with this family size variable, an easy way to set this up is now within_grp=True. Where we allow swaps just within each of the unique groups.

In [8]:
pvals, original_scores, h0_vmax = permuted_v(tested_vars=data['neurocog_pc1.bl'],
                                             target_vars=data[thick],
                                             confounding_vars=data['sex_at_birth'],
                                             permutation_structure=permutation_structure,
                                             within_grp=True,
                                             n_perm=1,
                                             n_jobs=4,
                                             verbose=1)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:   17.1s finished


### Make some plots of our results

In [ ]:
from neurotools.plotting import SurfRef, plot

def to_vertex(vals, names, space='fsaverage'):
    '''Helper function to convert destr. ROIs to vertex space'''
    
    plot_df = pd.DataFrame(vals, columns=['vals'])
    plot_df['names'] = names

    surf_ref = SurfRef(space=space, parc='destr')
    plot_data = surf_ref.get_hemis_plot_vals(plot_df, lh_key='.lh', rh_key='.rh')
    
    return plot_data

In [ ]:
roi_names = list(data[thick])
v = to_vertex(original_scores, roi_names)
plot(v, title='Raw v-stats', threshold=.1)

In [ ]:
neg_log_pvals = -np.log10(pvals)
v = to_vertex(neg_log_pvals , roi_names)
plot(v, title='Neg log10 pvals', threshold=.001)

In [ ]:
thresh_scores = original_scores.copy()
thresh_scores[pvals > .05] = 0

v = to_vertex(thresh_scores, roi_names)
plot(v, title='Thresholded V stats', threshold=.1)

What about for ML / Multi-variate?

In [58]:
from neurotools.random.permute_blocks import block_permutation
import BPt as bp

In [36]:
data = bp.Dataset(data)
data = data.set_role('neurocog', 'target')
data = data.set_role(['sex_at_birth', 'family_size', 'neg_rel_family_id', 'ones', 'rel_family_id'], 'non input')

subjs = data.index[:500]

In [37]:
results = bp.evaluate('ridge_pipe', data, subjects=subjs)

Predicting target = neurocog_pc1.bl
Using problem_type = regression
Using scope = all (defining a total of 148 features).
Evaluating 500 total data points.


Folds:   0%|          | 0/5 [00:00<?, ?it/s]


Training Set: (400, 148)
Validation Set: (100, 148)
Fit fold in 1.8 seconds.
explained_variance: 0.0829
neg_mean_squared_error: -0.6376

Training Set: (400, 148)
Validation Set: (100, 148)
Fit fold in 2.1 seconds.
explained_variance: 0.0343
neg_mean_squared_error: -0.4570

Training Set: (400, 148)
Validation Set: (100, 148)
Fit fold in 1.9 seconds.
explained_variance: 0.1044
neg_mean_squared_error: -0.4624

Training Set: (400, 148)
Validation Set: (100, 148)
Fit fold in 2.2 seconds.
explained_variance: 0.0585
neg_mean_squared_error: -0.6158

Training Set: (400, 148)
Validation Set: (100, 148)
Fit fold in 3.6 seconds.
explained_variance: 0.0839
neg_mean_squared_error: -0.5940



In [41]:
baseline_score = results.score
baseline_score

0.0728174110395607

In [68]:
n_perm = 10
p_scores = []
for repeat in range(n_perm):
    
    p_data = data.copy()

    # Permute the subjects randomly
    permuted_subjs = np.random.permutation(subjs)
    
    # Or permute according to restrictions
    permuted_subjs = block_permutation(subjs, blocks=permutation_structure.loc[subjs],
                                       within_grp=True)

    # Apply permutation to the target
    p_data.loc[subjs, 'neurocog_pc1.bl'] = np.array(p_data.loc[permuted_subjs, 'neurocog_pc1.bl'])

    p_results = bp.evaluate('ridge_pipe', p_data, subjects=subjs,
                            eval_verbose=-1, progress_bar=None)
    p_scores.append(p_results.score)